In [1]:
import gzip
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import netCDF4 as nc
import match
from netCDF4 import Dataset
import xarray as xra
import os 
import pandas as pd  
from datetime import datetime

In [4]:
folder_path = 'D:/data/SST/'
file_names = ['SST_2011-2012_month.nc','SST_2013-2014_month.nc','SST_2015-2016_month.nc',
             'SST_2017-2018_month.nc','SST_2019-2020_month.nc','SST_2021_month.nc']

In [8]:
file_new = []
for i in range(len(file_names)):
    SST = nc.Dataset((folder_path+file_names[i]))
    file_new.append(SST)

In [14]:
len(file_new)

6

In [15]:
SST0 = file_new[0]['sst'][:].data
SST1 = file_new[1]['sst'][:].data
SST2 = file_new[2]['sst'][:].data
SST3 = file_new[3]['sst'][:].data
SST4 = file_new[4]['sst'][:].data
SST5 = file_new[5]['sst'][:].data

In [17]:
print(SST2.shape)

(24, 3600, 7200)


In [18]:
SST_2011_2021 = np.concatenate((SST0,SST1,SST2,SST3,SST4,SST5),axis=0)

In [19]:
SST_2011_2021.shape 

(132, 3600, 7200)

In [31]:
SST_2011_2021_sheshidu = SST_2011_2021 - 273.15

In [33]:
np.nanmin(SST_2011_2021_sheshidu)

-1.9467773

# 保存合并数据

In [20]:
lat = file_new[0]['lat'][:].data
lon = file_new[0]['lon'][:].data

In [25]:
np.nanmin(SST_2011_2021)

271.20322

In [28]:
start_year = 2011
end_year =2021
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year,12,31),freq='1m')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [29]:
len(time)

132

In [34]:
#new_NC.close()
new_NC = Dataset("D:/data/SST/SST_2011-2021_month_sheshidu.nc", 'w', format='NETCDF4')

new_NC.createDimension('lat', len(lat))
new_NC.createDimension('lon', len(lon))
new_NC.createDimension('time', len(time))


time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2011-01-15'
time_var.long_name = 'months'
time_var.axis = 'T' 

new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))
new_NC.createVariable('sst', 'f',("time","lat","lon"))

#向变量中填充数据
new_NC.variables['lat'][:] = lat
new_NC.variables['lon'][:] = lon
new_NC.variables['time'][:] = np.array(time)
new_NC.variables['sst'][:]=np.array(SST_2011_2021_sheshidu)
 
#最后记得关闭文件
new_NC.close()



# 对数据进行线性插值

In [43]:
print(lat)
print(lon)

[-89.975 -89.925 -89.875 ...  89.875  89.925  89.975]
[-179.975 -179.925 -179.875 ...  179.875  179.925  179.975]


In [38]:
data_sst = xr.open_dataset('D:/data/SST/SST_2011-2021_month_sheshidu.nc',decode_times=False)

data_ssh = nc.Dataset(r'D:/data/SSH/2019/01/dt_global_twosat_phy_l4_20190101_vDT2021.nc')

In [40]:
sst = data_sst['sst']#读取sssa数据，读完后也是一个DataSet

In [42]:
#以下这一步就是插值的过程
lon_ssh = data_ssh.variables['longitude'][:].data
print(lon_ssh)
lat_ssh = data_ssh.variables['latitude'][:].data
print(lat_ssh)

[-179.875 -179.625 -179.375 ...  179.375  179.625  179.875]
[-89.875 -89.625 -89.375 -89.125 -88.875 -88.625 -88.375 -88.125 -87.875
 -87.625 -87.375 -87.125 -86.875 -86.625 -86.375 -86.125 -85.875 -85.625
 -85.375 -85.125 -84.875 -84.625 -84.375 -84.125 -83.875 -83.625 -83.375
 -83.125 -82.875 -82.625 -82.375 -82.125 -81.875 -81.625 -81.375 -81.125
 -80.875 -80.625 -80.375 -80.125 -79.875 -79.625 -79.375 -79.125 -78.875
 -78.625 -78.375 -78.125 -77.875 -77.625 -77.375 -77.125 -76.875 -76.625
 -76.375 -76.125 -75.875 -75.625 -75.375 -75.125 -74.875 -74.625 -74.375
 -74.125 -73.875 -73.625 -73.375 -73.125 -72.875 -72.625 -72.375 -72.125
 -71.875 -71.625 -71.375 -71.125 -70.875 -70.625 -70.375 -70.125 -69.875
 -69.625 -69.375 -69.125 -68.875 -68.625 -68.375 -68.125 -67.875 -67.625
 -67.375 -67.125 -66.875 -66.625 -66.375 -66.125 -65.875 -65.625 -65.375
 -65.125 -64.875 -64.625 -64.375 -64.125 -63.875 -63.625 -63.375 -63.125
 -62.875 -62.625 -62.375 -62.125 -61.875 -61.625 -61.375 -61.125

In [44]:
sst= sst.interp(lat=lat_ssh, lon=lon_ssh)

In [46]:
len(lon_ssh)

1440

# 保存插值之后的SST

In [49]:
len(time)

132

In [50]:
#new_NC.close()
new_NC = Dataset("D:/data/SST/SST_2011-2021_month_sheshidu_interp.nc", 'w', format='NETCDF4')

new_NC.createDimension('lat', len(lat_ssh))
new_NC.createDimension('lon', len(lon_ssh))
new_NC.createDimension('time', len(time))


time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2011-01-15'
time_var.long_name = 'months'
time_var.axis = 'T' 

new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))
new_NC.createVariable('sst', 'f',("time","lat","lon"))

#向变量中填充数据
new_NC.variables['lat'][:] = lat_ssh
new_NC.variables['lon'][:] = lon_ssh
new_NC.variables['time'][:] = np.array(time)
new_NC.variables['sst'][:]=np.array(sst)
 
#最后记得关闭文件
new_NC.close()

